In [1]:
import pandas as pd 
from cc_train_val_test import get_upsampled_dataset

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train,val,test,tokenizer = get_upsampled_dataset()

In [3]:
train_x,train_y = train
val_x, val_y = val
test_x,test_y = test

In [4]:
import keras as k
from keras.layers import Embedding,LSTM,Dense,Flatten,Activation,RepeatVector,Permute,multiply,Lambda, Input, merge
from keras.backend import sum
from keras import Model

In [18]:
vocab_size = len(tokenizer.index_word)+1
max_length = 15
units = 15
embedding_size = 100

In [31]:
_input = Input(shape=[max_length], dtype='int32')

# get the embedding layer
embedded = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_size,
        input_length=max_length,
        trainable=False,
        mask_zero=False
    )(_input)

activations = LSTM(units, return_sequences=True)(embedded)

# compute importance for each step
attention = Dense(1, activation='tanh')(activations)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(units)(attention)
attention = Permute([2, 1],name='attention')(attention)


sent_representation = multiply([activations, attention])
sent_representation = Lambda(lambda xin: sum(xin, axis=1), output_shape=(units,),name='sent_rep')(sent_representation)

probabilities = Dense(12, activation='softmax')(sent_representation)

In [32]:
model = Model(input=_input, output=probabilities)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [33]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 15, 100)      3919700     input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 15, 15)       6960        embedding_3[0][0]                
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 15, 1)        16          lstm_3[0][0]                     
__________________________________________________________________________________________________
flatten_3 

In [34]:
model.fit(train_x,train_y,validation_data=(val_x,val_y),batch_size=1024,)

Train on 177792 samples, validate on 7524 samples
Epoch 1/1
177792/177792 [==============================] - 9s 48us/step - loss: 2.3413 - categorical_accuracy: 0.1079 - val_loss: 1.9542 - val_categorical_accuracy: 0.0510


In [39]:
model.get_layer('sent_rep')

In [30]:
mod

AttributeError: 'Tensor' object has no attribute 'weights'